In [ ]:
Classe joueur
Noter chaque gate des joueurs.
Fonction get_gate() qui permet de savoir combien de chaque gate le joueur a de gate. Retourne dictionnaire Ex : H:3 = Hadamar pour 3 gate
Avec classe joueur = Nom + gates.
X,Y,Z,H

In [ ]:
%pip install -U pygame

In [ ]:
import pygame
import sys
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit_aer import AerSimulator

# --- Quantum Logic ---

def hadamard():
    return np.array([[1, 1], [1, -1]]) / np.sqrt(2)

def x_gate():
    return np.array([[0, 1], [1, 0]])

def y_gate():
    return np.array([[0, -1j], [1j, 0]])

def z_gate():
    return np.array([[1, 0], [0, -1]])

def cnot():
    return np.array([
        [1,0,0,0],
        [0,1,0,0],
        [0,0,0,1],
        [0,0,1,0]
    ])

def swap():
    return np.array([
        [1,0,0,0],
        [0,0,1,0],
        [0,1,0,0],
        [0,0,0,1]
    ])

GATE_MATRICES = {
    "H": hadamard(),
    "X": x_gate(),
    "Y": y_gate(),
    "Z": z_gate(),
    "CNOT": cnot(),
    "SWAP": swap()
}

# --- Player Class ---

class Player:
    def __init__(self, name):
        self.name = name
        self.gates = {"H":1, "Z":1, "Y":1, "X":1, "CNOT":1, "SWAP":1}
    def get_gate(self):
        return dict(self.gates)

# --- Game Setup ---

pygame.init()
WIDTH, HEIGHT = 900, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Quantum Gate Minigame")

font = pygame.font.SysFont(None, 32)

players = [Player("Mario"), Player("Luigi"), Player("Yoshi"), Player("Peach")]
current_player = 0
skipped_players = set()

# --- Gate UI Setup ---

GATE_COLORS = {
    "H": (200,200,255),
    "Z": (255,200,200),
    "Y": (200,255,200),
    "X": (255,255,200),
    "CNOT": (200,255,255),
    "SWAP": (255,200,255)
}
GATE_LIST = ["H","Z","Y","X","CNOT","SWAP"]

gate_rects = {}
for i, gate in enumerate(GATE_LIST):
    gate_rects[gate] = pygame.Rect(30, 50 + i*60, 80, 40)

# --- Circuit State ---
# For CNOT and SWAP, store both qubits: (gate, q1, q2)
circuit = []  # List of (gate, ...) tuples
MAX_GATES = 10

def draw_gates(player):
    for i, gate in enumerate(GATE_LIST):
        rect = gate_rects[gate]
        # Highlight in green if current player
        if players[current_player] == player:
            pygame.draw.rect(screen, (0, 255, 0), rect, 4)
        pygame.draw.rect(screen, GATE_COLORS[gate], rect)
        txt = font.render(f"{gate} ({player.gates[gate]})", True, (0,0,0))
        screen.blit(txt, (rect.x+5, rect.y+5))

def draw_circuit():
    base_x = 200
    base_y = 150
    place_x = base_x + (len(circuit)+1)*60
    for q in range(2):
        y = base_y + q*60
        pygame.draw.line(screen, (0,0,0), (base_x, y), (base_x+MAX_GATES*60, y), 2)
        pygame.draw.rect(screen, (0,255,0), (place_x-20, y-20, 40, 40), 3)
    # For CNOT/SWAP, highlight the area spanning both qubits
    pygame.draw.rect(screen, (0,255,0), (place_x-20, base_y-20, 40, 100), 3)
    for idx, gate_info in enumerate(circuit):
        x = base_x + (idx+1)*60
        if gate_info[0] == "CNOT":
            _, control, target = gate_info
            y1 = base_y + control*60
            y2 = base_y + target*60
            # Draw control dot
            pygame.draw.circle(screen, (0,0,0), (x, y1), 8)
            # Draw vertical line
            pygame.draw.line(screen, (0,0,0), (x, y1), (x, y2), 2)
            # Draw target (circle with plus)
            pygame.draw.circle(screen, (0,0,0), (x, y2), 12, 2)
            pygame.draw.line(screen, (0,0,0), (x-10, y2), (x+10, y2), 2)
            pygame.draw.line(screen, (0,0,0), (x, y2-10), (x, y2+10), 2)
        elif gate_info[0] == "SWAP":
            _, q1, q2 = gate_info
            y1 = base_y + q1*60
            y2 = base_y + q2*60
            # Draw X on both qubits
            pygame.draw.line(screen, (0,0,0), (x-10, y1-10), (x+10, y1+10), 2)
            pygame.draw.line(screen, (0,0,0), (x-10, y1+10), (x+10, y1-10), 2)
            pygame.draw.line(screen, (0,0,0), (x-10, y2-10), (x+10, y2+10), 2)
            pygame.draw.line(screen, (0,0,0), (x-10, y2+10), (x+10, y2-10), 2)
            # Draw vertical line
            pygame.draw.line(screen, (0,0,0), (x, y1), (x, y2), 2)
        else:
            gate, qubit = gate_info
            y = base_y + qubit*60
            pygame.draw.rect(screen, GATE_COLORS[gate], (x-20, y-20, 40, 40))
            txt = font.render(gate, True, (0,0,0))
            screen.blit(txt, (x-10, y-10))

def draw_measure_button():
    btn_rect = pygame.Rect(WIDTH-180, HEIGHT-80, 150, 50)
    pygame.draw.rect(screen, (100,200,100), btn_rect)
    txt = font.render("Measurement", True, (0,0,0))
    screen.blit(txt, (btn_rect.x+10, btn_rect.y+10))
    return btn_rect

def draw_skip_button():
    btn_rect = pygame.Rect(WIDTH-350, HEIGHT-80, 120, 50)
    pygame.draw.rect(screen, (200,100,100), btn_rect)
    txt = font.render("Skip", True, (0,0,0))
    screen.blit(txt, (btn_rect.x+30, btn_rect.y+10))
    return btn_rect

def draw_player_info():
    txt = font.render(f"Current Player: {players[current_player].name}", True, (0,0,0))
    screen.blit(txt, (30, 10))

# --- Drag and Drop ---
dragging_gate = None
drag_offset = (0,0)
drag_pos = (0,0)

# --- Main Loop ---

measurement_result = None

def next_player():
    global current_player
    n = len(players)
    for _ in range(n):
        current_player = (current_player + 1) % n
        if current_player not in skipped_players:
            return

while True:
    screen.fill((240,240,240))
    draw_player_info()
    draw_gates(players[current_player])
    draw_circuit()
    btn_rect = draw_measure_button()
    skip_btn_rect = draw_skip_button()
    if measurement_result:
        txt = font.render(f"Measured: {measurement_result}", True, (0,0,0))
        screen.blit(txt, (WIDTH-300, HEIGHT-140))
    # Draw dragging gate if any
    if dragging_gate:
        mx, my = drag_pos
        pygame.draw.rect(screen, GATE_COLORS[dragging_gate], (mx-40, my-20, 80, 40))
        txt = font.render(dragging_gate, True, (0,0,0))
        screen.blit(txt, (mx-20, my-10))
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = event.pos
            if skip_btn_rect.collidepoint(mx, my):
                skipped_players.add(current_player)
                next_player()
                dragging_gate = None
                drag_pos = (0,0)
                continue
            for gate, rect in gate_rects.items():
                if rect.collidepoint(mx, my) and players[current_player].gates[gate] > 0:
                    dragging_gate = gate
                    drag_offset = (mx - rect.x, my - rect.y)
                    drag_pos = (mx, my)
            if btn_rect.collidepoint(mx, my):
                measurement_result = apply_circuit()
        elif event.type == pygame.MOUSEBUTTONUP:
            if dragging_gate:
                mx, my = event.pos
                base_x = 200 + (len(circuit)+1)*60
                dropped = False
                if dragging_gate == "CNOT":
                    for q in range(2):
                        y = 150 + q*60
                        if base_x-20 < mx < base_x+20 and y-20 < my < y+20:
                            control = q
                            target = 1 - q
                            circuit.append(("CNOT", control, target))
                            players[current_player].gates["CNOT"] -= 1
                            next_player()
                            measurement_result = None
                            dropped = True
                            break
                    dragging_gate = None
                    drag_pos = (0,0)
                elif dragging_gate == "SWAP":
                    # SWAP: always between 0 and 1
                    circuit.append(("SWAP", 0, 1))
                    players[current_player].gates["SWAP"] -= 1
                    next_player()
                    measurement_result = None
                    dragging_gate = None
                    drag_pos = (0,0)
                else:
                    for q in range(2):
                        y = 150 + q*60
                        if base_x-20 < mx < base_x+20 and y-20 < my < y+20:
                            circuit.append((dragging_gate, q))
                            players[current_player].gates[dragging_gate] -= 1
                            next_player()
                            measurement_result = None
                            dropped = True
                            break
                    dragging_gate = None
                    drag_pos = (0,0)
        elif event.type == pygame.MOUSEMOTION:
            if dragging_gate:
                drag_pos = event.pos

# --- Quantum Circuit Logic ---
def apply_circuit():
    # 2 qubits, 2 classical bits for measurement
    qc = QuantumCircuit(2, 2)
    # Start with both qubits in |0&gt;
    # Apply H to both as in original logic
    qc.h(0)
    qc.h(1)
    for gate_info in circuit:
        if gate_info[0] in ["H", "X", "Y", "Z"]:
            gate, qubit = gate_info
            if gate == "H":
                qc.h(qubit)
            elif gate == "X":
                qc.x(qubit)
            elif gate == "Y":
                qc.y(qubit)
            elif gate == "Z":
                qc.z(qubit)
        elif gate_info[0] == "CNOT":
            _, control, target = gate_info
            qc.cx(control, target)
        elif gate_info[0] == "SWAP":
            _, q1, q2 = gate_info
            qc.swap(q1, q2)
    # Measure both qubits
    qc.measure(0, 0)
    qc.measure(1, 1)
    # Simulate
    sim = AerSimulator()
    result = sim.run(qc).result()
    counts = result.get_counts()
    # Return the most probable result
    measured = max(counts, key=counts.get)
    return measured

In [ ]:
# Minimal pygame quantum circuit game prototype

import pygame
from qiskit import QuantumCircuit
import numpy as np

# Add missing constants and initialization
PLAYER_COLORS = [(255, 99, 71), (65, 105, 225)]
BG_COLOR = (245, 245, 255)
CIRCUIT_BG = (220, 220, 235)
CIRCUIT_BORDER = (120, 120, 180)
WIDTH = 1200

pygame.init()
FONT = pygame.font.SysFont("arial", 32)
SMALLFONT = pygame.font.SysFont("arial", 20)
screen = pygame.display.set_mode((WIDTH, 700))

class Joueur:
    def __init__(self, nom, gates):
        self.nom = nom
        self.gates = gates  # dict: gate -> count

    def get_gate(self):
        return self.gates.copy()

    def use_gate(self, gate):
        if self.gates.get(gate, 0) > 0:
            self.gates[gate] -= 1
            return True
        return False

# Ajout des nouvelles portes et initialisation
ALL_GATES = {"h": "Hadamard", "y": "Y", "z": "Z", "cnot": "CNOT", "swap": "SWAP"}
GATE_COLORS = {"h": (255, 215, 0), "y": (135, 206, 250), "z": (186, 85, 211), "cnot": (255, 140, 0), "swap": (60, 179, 113)}

joueurs = [
    Joueur("julien", {"h": 1, "y": 3, "z": 6, "cnot": 1, "swap": 1}),
    Joueur("cassandre", {"h": 2, "y": 1, "z": 1, "cnot": 1, "swap": 1}),
]

# circuit: list of (gate, player_name, qubit_idx, [target_qubit])
# Pour CNOT et SWAP, on stocke la cible dans target_qubit
circuit = []

NUM_QUBITS = 2
QUBIT_LABELS = [f"q[{i}]" for i in range(NUM_QUBITS)]

# Drag state
dragging = False
drag_gate = None
drag_player = None
drag_offset = (0, 0)
drag_gate_idx = None
drag_target_qubit = None  # Pour CNOT/SWAP

# Pour bouton calcul
calc_button_rect = pygame.Rect(1020, 600, 160, 50)
last_result = None

def draw_rounded_rect(surf, color, rect, radius=12, border=0, border_color=(0,0,0)):
    pygame.draw.rect(surf, color, rect, border_radius=radius)
    if border > 0:
        pygame.draw.rect(surf, border_color, rect, border, border_radius=radius)

def draw_player_gates():
    global gate_icons
    gate_icons = []
    y = 40
    for idx, joueur in enumerate(joueurs):
        x = 40
        color = PLAYER_COLORS[idx % len(PLAYER_COLORS)]
        name_surf = FONT.render(joueur.nom.capitalize(), True, color)
        screen.blit(name_surf, (x, y))
        x += 200
        for gate, count in joueur.get_gate().items():
            if count > 0:
                gate_rect = pygame.Rect(x, y, 60, 60)
                draw_rounded_rect(screen, GATE_COLORS.get(gate, (200,200,200)), gate_rect, radius=16, border=3, border_color=color)
                gate_text = FONT.render(gate.upper(), True, (40,40,40))
                screen.blit(gate_text, (x+8, y+10))
                count_text = SMALLFONT.render(f"{count}x", True, (60,60,60))
                screen.blit(count_text, (x+10, y+40))
                gate_icons.append((gate_rect, gate, idx))
                x += 90
        y += 100

def draw_circuit():
    # --- UI constants for image-like style ---
    QUBIT_Y = [350, 410]  # y positions for q[0], q[1]
    CLASSIC_Y = 470
    X_START = 170
    X_GAP = 90
    N_STEPS = 10

    # Colors for gates
    X_COLOR = (40, 80, 200)
    H_COLOR = (220, 60, 80)
    MEAS_COLOR = (180, 180, 180)
    CNOT_COLOR = (40, 80, 200)
    BARRIER_COLOR = (120, 120, 120)
    LINE_COLOR = (80, 80, 80)

    # Draw circuit area background
    circuit_rect = pygame.Rect(80, 300, 900, 200)
    draw_rounded_rect(screen, CIRCUIT_BG, circuit_rect, radius=18, border=4, border_color=CIRCUIT_BORDER)

    # Draw horizontal lines for qubits
    for i, y in enumerate(QUBIT_Y):
        pygame.draw.line(screen, LINE_COLOR, (X_START-60, y), (X_START + X_GAP*N_STEPS, y), 2)
        label = SMALLFONT.render(f"q[{i}]", True, (40,40,40))
        screen.blit(label, (X_START-110, y-10))

    # Draw classical bit line
    pygame.draw.line(screen, (120,120,120), (X_START-60, CLASSIC_Y), (X_START + X_GAP*N_STEPS, CLASSIC_Y), 2)
    screen.blit(SMALLFONT.render("meas", True, (40,40,40)), (X_START-110, CLASSIC_Y-10))

    # Build a list of gates per column (step)
    steps = []
    for gate_tuple in circuit:
        placed = False
        for i, step in enumerate(steps):
            involved = [gate_tuple[2]]
            if gate_tuple[0] in ("cnot", "swap") and len(gate_tuple) > 3:
                involved.append(gate_tuple[3])
            if all(step.get(q) is None for q in involved):
                for q in involved:
                    step[q] = gate_tuple
                placed = True
                break
        if not placed:
            step = {q: None for q in range(NUM_QUBITS)}
            involved = [gate_tuple[2]]
            if gate_tuple[0] in ("cnot", "swap") and len(gate_tuple) > 3:
                involved.append(gate_tuple[3])
            for q in involved:
                step[q] = gate_tuple
            steps.append(step)

    # Draw gates column by column
    for col, step in enumerate(steps):
        for q in range(NUM_QUBITS):
            gate_tuple = step.get(q)
            if gate_tuple is None:
                continue
            gate, player, qubit, *target = gate_tuple
            x = X_START + col * X_GAP
            y = QUBIT_Y[q]
            # Draw gates as in the image
            if gate == "x":
                rect = pygame.Rect(x-20, y-20, 40, 40)
                pygame.draw.rect(screen, X_COLOR, rect, border_radius=8)
                screen.blit(FONT.render("X", True, WHITE), (rect.x+12, rect.y+8))
            elif gate == "h":
                rect = pygame.Rect(x-20, y-20, 40, 40)
                pygame.draw.rect(screen, H_COLOR, rect, border_radius=8)
                screen.blit(FONT.render("H", True, WHITE), (rect.x+10, rect.y+8))
            elif gate == "cnot" and target:
                q2 = target[0]
                y2 = QUBIT_Y[q2]
                # Draw control dot
                pygame.draw.circle(screen, CNOT_COLOR, (x, y), 7)
                # Draw vertical line
                pygame.draw.line(screen, CNOT_COLOR, (x, y), (x, y2), 2)
                # Draw target plus
                pygame.draw.circle(screen, WHITE, (x, y2), 13)
                pygame.draw.circle(screen, CNOT_COLOR, (x, y2), 13, 2)
                pygame.draw.line(screen, CNOT_COLOR, (x, y2-13), (x, y2+13), 2)
                pygame.draw.line(screen, CNOT_COLOR, (x-13, y2), (x+13, y2), 2)
            elif gate == "z":
                rect = pygame.Rect(x-20, y-20, 40, 40)
                pygame.draw.rect(screen, (120, 60, 180), rect, border_radius=8)
                screen.blit(FONT.render("Z", True, WHITE), (rect.x+10, rect.y+8))
            elif gate == "y":
                rect = pygame.Rect(x-20, y-20, 40, 40)
                pygame.draw.rect(screen, (60, 180, 180), rect, border_radius=8)
                screen.blit(FONT.render("Y", True, WHITE), (rect.x+10, rect.y+8))
            elif gate == "swap" and target:
                q2 = target[0]
                y2 = QUBIT_Y[q2]
                for yy in [y, y2]:
                    pygame.draw.line(screen, (0,0,0), (x-10, yy-10), (x+10, yy+10), 3)
                    pygame.draw.line(screen, (0,0,0), (x+10, yy-10), (x-10, yy+10), 3)
            elif gate == "meas":
                rect = pygame.Rect(x-20, y-20, 40, 40)
                pygame.draw.rect(screen, MEAS_COLOR, rect, border_radius=8)
                screen.blit(FONT.render("M", True, BLACK), (rect.x+8, rect.y+8))
                # Draw arrow to classical bit line
                arrow_start = (rect.centerx, rect.bottom)
                arrow_end = (rect.centerx, CLASSIC_Y)
                pygame.draw.line(screen, (120,120,120), arrow_start, arrow_end, 2)
                pygame.draw.polygon(screen, (120,120,120), [
                    (arrow_end[0]-5, arrow_end[1]-5),
                    (arrow_end[0]+5, arrow_end[1]-5),
                    (arrow_end[0], arrow_end[1]+8)
                ])
                # Draw measurement result label
                screen.blit(SMALLFONT.render("0", True, BLACK), (arrow_end[0]-5, CLASSIC_Y+10))
            # Optionally, draw barrier as dashed vertical line
            if gate == "barrier":
                bx = x
                for yy in range(QUBIT_Y[0]-15, QUBIT_Y[-1]+16, 8):
                    pygame.draw.line(screen, BARRIER_COLOR, (bx, yy), (bx, yy+4), 2)

    # Draw hint if dragging
    if dragging:
        drop_hint = SMALLFONT.render("Glisser une porte sur une ligne Qubit", True, (120,120,120))
        screen.blit(drop_hint, (circuit_rect.centerx-90, circuit_rect.bottom+10))

def draw_dragged_gate():
    if dragging and drag_gate is not None:
        mx, my = pygame.mouse.get_pos()
        color = GATE_COLORS.get(drag_gate, (200,200,200))
        player_color = PLAYER_COLORS[drag_player % len(PLAYER_COLORS)]
        gate_rect = pygame.Rect(mx+drag_offset[0], my+drag_offset[1], 60, 60)
        draw_rounded_rect(screen, color, gate_rect, radius=16, border=3, border_color=player_color)
        if drag_gate == "cnot":
            gate_text = FONT.render("CX", True, (40,40,40))
            screen.blit(gate_text, (mx+drag_offset[0]+5, my+drag_offset[1]+22))
        elif drag_gate == "swap":
            pygame.draw.line(screen, (0,0,0), (mx+drag_offset[0]+15, my+drag_offset[1]+15), (mx+drag_offset[0]+45, my+drag_offset[1]+45), 3)
            pygame.draw.line(screen, (0,0,0), (mx+drag_offset[0]+45, my+drag_offset[1]+15), (mx+drag_offset[0]+15, my+drag_offset[1]+45), 3)
            gate_text = FONT.render("SW", True, (40,40,40))
            screen.blit(gate_text, (mx+drag_offset[0]+5, my+drag_offset[1]+22))
        else:
            gate_text = FONT.render(drag_gate.upper(), True, (40,40,40))
            screen.blit(gate_text, (mx+drag_offset[0]+18, my+drag_offset[1]+10))

def get_qubit_under_mouse(pos):
    x, y = pos
    y_start = 350
    y_gap = 60
    for q in range(NUM_QUBITS):
        line_y = y_start + q * y_gap + 30
        if 130 <= x <= 130 + 80*10 and (line_y-30) <= y <= (line_y+30):
            return q
    return None

def draw_calc_button():
    draw_rounded_rect(screen, (100, 200, 100), calc_button_rect, radius=14, border=3, border_color=(0, 100, 0))
    txt = FONT.render("Calcul état final", True, (0, 60, 0))
    screen.blit(txt, (calc_button_rect.x+10, calc_button_rect.y+8))

def draw_result():
    if last_result:
        txt = SMALLFONT.render("Probabilités (états):", True, (0,0,0))
        screen.blit(txt, (1020, 540))
        for i, (state, prob) in enumerate(last_result.items()):
            s = SMALLFONT.render(f"{state}: {prob:.3f}", True, (0,0,0))
            screen.blit(s, (1020, 560 + 20*i))

def build_and_run_circuit():
    qc = QuantumCircuit(NUM_QUBITS)
    for gate, player, qubit, *target in circuit:
        if gate == "h":
            qc.h(qubit)
        elif gate == "y":
            qc.y(qubit)
        elif gate == "z":
            qc.z(qubit)
        elif gate == "cnot" and target:
            qc.cx(qubit, target[0])
        elif gate == "swap" and target:
            qc.swap(qubit, target[0])
    qc.save_statevector()
    backend = Aer.get_backend('aer_simulator')
    result = backend.run(qc).result()
    statevector = result.get_statevector()
    probs = np.abs(statevector) ** 2
    states = [format(i, f'0{NUM_QUBITS}b') for i in range(2**NUM_QUBITS)]
    return dict(zip(states, probs))

def mainloop():
    global dragging, drag_gate, drag_player, drag_offset, drag_gate_idx, drag_target_qubit, last_result
    global screen  # <-- Move this to the top of the function
    clock = pygame.time.Clock()
    selecting_target = False
    source_qubit = None
    running = True

    # Ensure pygame is initialized and display is set before entering the loop
    if not pygame.get_init():
        pygame.init()
    if not pygame.display.get_init():
        pygame.display.init()
    try:
        screen.get_size()
    except pygame.error:
        # If screen is not initialized, re-create it
        screen = pygame.display.set_mode((1200, 700))

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                break
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mx, my = event.pos
                if calc_button_rect.collidepoint(mx, my):
                    last_result = build_and_run_circuit()
                elif not selecting_target:
                    for idx, (rect, gate, player_idx) in enumerate(gate_icons):
                        if rect.collidepoint(mx, my):
                            dragging = True
                            drag_gate = gate
                            drag_player = player_idx
                            drag_offset = (rect.x - mx, rect.y - my)
                            drag_gate_idx = idx
                            break
            elif event.type == pygame.MOUSEBUTTONUP:
                if dragging:
                    mx, my = event.pos
                    qubit = get_qubit_under_mouse((mx, my))
                    if qubit is not None:
                        joueur = joueurs[drag_player]
                        if joueur.get_gate().get(drag_gate, 0) > 0:
                            if drag_gate in ("cnot", "swap"):
                                selecting_target = True
                                source_qubit = qubit
                            else:
                                circuit.append((drag_gate, joueur.nom, qubit))
                                joueur.use_gate(drag_gate)
                    dragging = False
                    drag_gate = None
                    drag_player = None
                    drag_offset = (0, 0)
                    drag_gate_idx = None
            elif event.type == pygame.MOUSEBUTTONDOWN and selecting_target:
                mx, my = event.pos
                target_qubit = get_qubit_under_mouse((mx, my))
                if target_qubit is not None and target_qubit != source_qubit:
                    joueur = joueurs[drag_player]
                    if joueur.get_gate().get(drag_gate, 0) > 0:
                        circuit.append((drag_gate, joueur.nom, source_qubit, target_qubit))
                        joueur.use_gate(drag_gate)
                    selecting_target = False
                    source_qubit = None

        if not running:
            break
        try:
            screen.fill(BG_COLOR)
            title = FONT.render("Quantum Circuit Game", True, (60,60,120))
            screen.blit(title, (WIDTH//2 - title.get_width()//2, 10))
            draw_player_gates()
            draw_circuit()
            draw_dragged_gate()
            draw_calc_button()
            draw_result()
            pygame.display.flip()
            clock.tick(60)
        except pygame.error:
            break
    pygame.quit()


c:\Users\gable\anaconda3\envs\IQuCodeFest\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


c:\Users\gable\anaconda3\envs\IQuCodeFest\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\gable\anaconda3\envs\IQuCodeFest\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'WHITE' is not defined